## Optimizing and Deploying AI Models with Pruna and Hugging Face

Objective: Build a complete tutorial demonstrating how to optimize the [Efficient-Large-Model/Sana_600M_1024px_ControlNet_HED](https://huggingface.co/Efficient-Large-Model/Sana_600M_1024px_ControlNet_HED) diffusion model using Pruna and deploy it seamlessly to the Hugging Face Hub.

Model: [Efficient-Large-Model/Sana_600M_1024px_ControlNet_HED](https://huggingface.co/Efficient-Large-Model/Sana_600M_1024px_ControlNet_HED)

Dataset: [data-is-better-together/open-image-preferences-v1-binarized](https://huggingface.co/datasets/data-is-better-together/open-image-preferences-v1-binarized)

To follow along, ensure that you have the Pruna SDK installed along with all required third-party libraries. Running this tutorial in a clean virtual environment is recommended for a smooth setup.

In [ ]:
pip install pruna 

In [ ]:
pip install datasets huggingface_hub gradio diffusers

You will need to login on the Hugging Face Hub for using the model weights. Run the cell below to do the same.

In [1]:
from huggingface_hub import notebook_login

notebook_login()


Token has not been saved to git credential helper.


### Smash Configuration

To optimize the model, we first need to define the methods that will enhance its performance. For detailed options, refer to the [SmashConfig guide](https://docs.pruna.ai/en/stable/docs_pruna/user_manual/configure.html).

In this tutorial, we will:

* Select a **quantizer** to reduce memory usage
* Use a **cacher** to store intermediate computation results, accelerating future operations
* Upload the optimized (smashed) model to the Hugging Face Hub for easy access and deployment

In [ ]:
import torch
from diffusers import FluxPipeline
from pruna import smash, SmashConfig

# 1. Load the original FLUX pipeline
pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16
).to("cuda")

# 2. Configure Pruna smash
smash_config = SmashConfig()
smash_config["quantizer"] = "hqq_diffusers" 
smash_config["cacher"] = "pab"

# 3. Smash the model
smashed_pipe = smash(model=pipe, smash_config=smash_config)

# 4. Push smashed pipeline to the Hub
smashed_pipe.save_to_hub("AINovice2005/smashed-FLUX.1-schnell-pruna")


### Load Dataset

In this step, we will load the dataset required for optimizing and evaluating the model. This dataset will serve as input data during the evaluation and help assess the model’s performance after applying quantization.

We will use the [`data-is-better-together/open-image-preferences-v1-binarized`](https://huggingface.co/datasets/data-is-better-together/open-image-preferences-v1-binarized) dataset, which contains binarized user image preferences. Loading the dataset correctly ensures that the input pipeline is ready for smooth optimization and deployment workflows.

In [ ]:
from datasets import load_dataset

# load the binarized Open Image Preferences prompts
ds = load_dataset("data-is-better-together/open-image-preferences-v1-binarized", split="train")

# preview 10 examples
for example in ds.select(range(50)):
    print(example["prompt"])


### Evaluate the Model

Now that the model and dataset are set up, we can proceed to evaluate the model using the **Pruna Evaluation Agent**. This evaluation helps us measure the model’s current performance before optimization, providing a baseline for comparison. It assesses how well the model performs on the given dataset and generates relevant metrics that will guide us in understanding the impact of our optimization configurations later.

In [11]:
from diffusers import DiffusionPipeline
from pruna import PrunaModel
from pruna.data.pruna_datamodule import PrunaDataModule
from pruna.evaluation.evaluation_agent import EvaluationAgent
from pruna.evaluation.task import Task

# Step 1: Load the smashed FLUX model from Hugging Face
pipe = DiffusionPipeline.from_pretrained("AINovice2005/smashed-FLUX.1-schnell-pruna")
pipe.to("cuda")  # or "cpu"

# Step 2: Save to local directory
save_path = "saved_model"
pipe.save_pretrained(save_path)

# Step 3: Load the saved model using PrunaModel.from_pretrained
smashed_model = PrunaModel.from_pretrained(model_path=save_path)

# Step 4: Set up evaluation task
metrics = ["clip_score", "psnr"]
datamodule = PrunaDataModule.from_string("data-is-better-together/open-image-preferences-v1-binarized")
task = Task(metrics, datamodule=datamodule, device="cuda")  # or "cpu"

# Step 5: Run evaluation
eval_agent = EvaluationAgent(task)
results = eval_agent.evaluate(smashed_model)

# Step 6: Print results
print(results.results)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Pipeline <class 'diffusers.pipelines.flux.pipeline_flux.FluxPipeline'> expected ['scheduler', 'text_encoder', 'text_encoder_2', 'tokenizer', 'tokenizer_2', 'transformer', 'vae'], but only {'vae', 'tokenizer', 'text_encoder_2', 'text_encoder', 'scheduler', 'tokenizer_2'} were passed.

### Gradio Demo

Once the model has been optimized, we can deploy the smashed model using **Gradio** to create an interactive demo. This allows anyone to test the model’s capabilities directly in their browser.

In this section, we will:

* Show how to deploy the optimized model on the Hugging Face Hub with a Gradio demo
* Discuss considerations such as **handling queuing**, especially if multiple users access the demo simultaneously
* Highlight best practices for integrating Gradio demos in your Hugging Face Space to ensure a smooth and responsive user experience

Creating a Gradio demo not only showcases your optimized model effectively but also enables easy sharing and real-world testing by the community.

In [ ]:
import gradio as gr
import torch
from diffusers import DiffusionPipeline


model_path = "AINovice2005/smashed-FLUX.1-schnell-pruna"
pipe = DiffusionPipeline.from_pretrained(model_path)
pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Inference function
def generate_image(prompt):
    result = pipe(prompt, num_inference_steps=25, guidance_scale=7.5)
    return result.images[0]

# Create Gradio interface
demo = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(lines=2, placeholder="Enter your prompt here...", label="Prompt"),
    outputs=gr.Image(type="pil"),
    title="FLUX Smashed Text-to-Image",
    description="Generate high-quality images using a smashed FLUX model optimized with Pruna."
)

# Launch the app
if __name__ == "__main__":
    demo.launch()


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Pipeline <class 'diffusers.pipelines.flux.pipeline_flux.FluxPipeline'> expected ['scheduler', 'text_encoder', 'text_encoder_2', 'tokenizer', 'tokenizer_2', 'transformer', 'vae'], but only {'vae', 'tokenizer', 'text_encoder_2', 'text_encoder', 'scheduler', 'tokenizer_2'} were passed.